In [ ]:
import torch
import non_local_boxes
import numpy as np

import warnings
warnings.filterwarnings("ignore")   # to ignore the warning messages

# Sugar coating for reloading
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
non_local_boxes.evaluate.nb_columns = 1

---
# I. Definitions

### Nonlocal Boxes

In [ ]:
PR = non_local_boxes.utils.PR
PRprime = non_local_boxes.utils.PRprime
P0 = non_local_boxes.utils.P_0
P1 = non_local_boxes.utils.P_1
SR = (P0+P1)/2
I = non_local_boxes.utils.I

P_NL = non_local_boxes.utils.P_NL
P_L = non_local_boxes.utils.P_L

Boxes_all = []  # it will contain all the extremal boxes of NS
for mu in range(2):
    for nu in range(2):
        for sigma in range(2):
            Boxes_all.append([P_NL(mu, nu, sigma), "PNL("+str(mu)+str(nu)+str(sigma)+")"])
            for tau in range(2):
                Boxes_all.append([P_L(mu, nu, sigma, tau), "PL("+str(mu)+str(nu)+str(sigma)+str(tau)+")"])

R_all = [Boxes_all[j][0] for j in range(len(Boxes_all))]

### Find the Multiplication Table (Gradient Descent)

In [ ]:
def projection(T):
    T = torch.max(T, torch.zeros_like(T))
    c = torch.sum(T)
    if c>1:  T += -torch.ones_like(T)*(c-1)/T.size(dim=0)
    return T

def loss(T, Q, R):
    # R is a list of 4x4 matrices
    # T is a nx1 tensor
    n = len(T)
    NewBox = torch.zeros_like(R[0])
    NewBox = torch.tensordot(T, R[:-1,:,:], dims=1)
    NewBox += (1-torch.sum(T))*R[n]
    return torch.sum(torch.abs(NewBox - Q))

def find_coeff(Q, R, learning_rate, number_steps, do_a_projection=True):
    # Goal: find (a_0, ..., a_{n-1}) such that Q = a_0*R_0 + ... + a_{n-1}*_R_{n-1} + (1-a_0-...-a_{n-1})*R_n
    # How: gradient descent
    # Q is 4x4 matrices
    # R is a list of 4x4 matrices
    # n := len(R)-1
    # T will be torch.tensor([a_0, ..., a_{n-1}])

    T = 0.5 * torch.ones(len(R) - 1)
    T.requires_grad=True
    for i in range(number_steps):
        loss(T, Q, R).backward()
        if do_a_projection:   T = projection(T - T.grad*learning_rate/(i+1)).detach()
        else:   T = (T - T.grad*learning_rate/(i+1)).detach()
        T.requires_grad = True
    return T

In [ ]:
def write_combinaison(P, Pname, Boxes, learning_rate=1, number_steps=5000, decimals=2, do_a_projection=True, print_mistakes=False):
    R = torch.zeros((len(Boxes), 4, 4))
    for j in range(len(Boxes)):
        R[j, :, :]=torch.tensor(Boxes[j][0]).clone().detach()
    string = Pname+" = "

    T = find_coeff(P.clone(), R, learning_rate, number_steps, do_a_projection)
    values = np.around(T.tolist(), decimals=decimals).tolist()
    values.append(1-sum(values))

    for j in range(len(values)):
        if values[j] != 0:
            if string != Pname+" = ": string += " + "
            if values[j]==1: string += Boxes[j][1]
            else: string += str(values[j])+"·"+Boxes[j][1]
    error = float(loss(torch.tensor(values[:-1]),P,R))
    if error == 0:   string += "    (exact)"
    else:      
        string += "    !!!!! ERROR="+str(error)+" !!!!!"
        if print_mistakes:   print(P)

    print(string)

In [ ]:
def multiplication_table(Boxes, W, learning_rate=1, number_steps=5000, decimals=2, do_a_projection=True):
    if non_local_boxes.evaluate.nb_columns!=1:
        print("\n   WARNING: Please, set the number of columns to 1 (in `non_local_boxes.evaluate`).")
        return None
    for Q1 in Boxes:
        print("\n-----\n")
        for Q2 in Boxes:
            P = non_local_boxes.utils.tensor_to_matrix(non_local_boxes.evaluate.R(W, Q1[0], Q2[0]))
            Pname = Q1[1]+" ⊠_W "+Q2[1]
            write_combinaison(P, Pname, Boxes, learning_rate, number_steps, decimals, do_a_projection)
            

---
# II. Draw the Multiplication Tables

### Multiplication Table of $\mathsf{W}_{bs}$:

In [ ]:
m=non_local_boxes.evaluate.nb_columns

Boxes = [
    [PR, "PR"],
    [SR, "SR"],
    [I, "I"]
    ]
W = non_local_boxes.utils.W_BS09(m).detach()

multiplication_table(
    Boxes=Boxes,
    W = W,
    learning_rate=1,   #1
    number_steps=5000, #5000
    decimals = 2,      #2
    do_a_projection = False
    )

### Multiplication Table of $\mathsf{W}_{\oplus}$:

In [ ]:
Boxes = [
    [PR, "PR"],
    [P0, "P0"],
    [P1, "P1"],
    [I, "I"]
    ]

W = torch.tensor([0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0])*1.
W = torch.t(W.repeat(m, 1))

multiplication_table(
    Boxes=Boxes,
    W = W,
    learning_rate=1,   #1
    number_steps=5000, #5000
    decimals = 2,      #2
    do_a_projection = False
    )

### Find a specific combination:

In [ ]:
Boxes = [
    [PR, "PR"],
    [P_L(0,0,0,0), "P0"],
    [P_L(0,1,0,1), "P1"],
    [P_L(0,0,0,1), "P01"],
    [P_L(0,1,0,0), "P10"]
    ]

W = non_local_boxes.utils.W_NSSRRB22(1).detach()
P =non_local_boxes.utils.tensor_to_matrix(non_local_boxes.evaluate.R(W, I, I))
Pname = "P"

write_combinaison(
    P=P,
    Pname = Pname,
    Boxes=Boxes,
    learning_rate=1,        # 1
    number_steps=int(5e5),  # int(1e5)
    decimals = 4,           # 4
    do_a_projection = False,
    print_mistakes = True
    )

---
# III. Given a slice, find a stabilizing wiring

We list some collapsing wirings in `non_local_boxes.utils.known_collapsing_W`. But not all of them stabilize a given slice of boxes: it may be that, for some box $\mathtt{P}$ in a slice of $\mathcal{NS}$, the box $\mathtt{P}\boxtimes_{\mathsf{W}}\mathtt{P}$ does not belong to that slice.

The purpose of the algorithm below is to check "how far" is a wiring to be stabilizing. If the sum of error is `0.0`, then the wiring stabilizes the slice. Otherwise, as the number increases, the wiring is less and less stabilizing.

In [ ]:
def find_stabilizing_wiring(Boxes, learning_rate=1, number_steps=5000, decimals=2, do_a_projection=True):
    Boxes_string = ""
    for P in Boxes:
        Boxes_string+=P[1]+"  "
    print("SLICE:  "+Boxes_string+"\n")
    print("Learning rate:       "+str(learning_rate))
    print("Number of steps:     "+str(number_steps))
    print("Number of decimals:  "+str(decimals)+"\n")
    i=0
    minvalue = 1000
    imin = 0
    assert(non_local_boxes.evaluate.nb_columns==1)
    print("WIRINGS  | Sum of errors")
    print("------------------------")
    for W in non_local_boxes.utils.known_collapsing_W:
        i+=1
        W = torch.tensor(W)*1.
        W = torch.t(W.repeat(1, 1))
        sum_errors=0

        for Q1 in Boxes:
            for Q2 in Boxes:
                P = non_local_boxes.utils.tensor_to_matrix(non_local_boxes.evaluate.R(W, Q1[0], Q2[0]))
                
                R = torch.zeros((len(Boxes), 4, 4))
                for j in range(len(Boxes)):
                    R[j, :, :]=torch.tensor(Boxes[j][0]).clone().detach()   

                T = find_coeff(P.clone(), R, learning_rate, number_steps, do_a_projection)
                values = np.around(T.tolist(), decimals=decimals).tolist()
                values.append(1-sum(values))
                error = float(loss(torch.tensor(values[:-1]),P,R))
                sum_errors+=error

        string="W_"+str(i)
        for _ in range(int(6-np.floor(np.log10(i)))):
            string+=" "
        string+="| "+str(sum_errors)
        print(string)
        if sum_errors<minvalue:
            minvalue=sum_errors
            imin=i
    
    print("\nMinimal value: "+str(minvalue)+",  achieved at W_"+str(imin))

In [ ]:
Boxes = [
    [PR, "PR"],
    [P_L(0,0,0,0), "P0"],
    [P_L(0,1,0,1), "P1"],
    ]

find_stabilizing_wiring(
    Boxes=Boxes,
    learning_rate=1,   #1
    number_steps=5000, #5000
    decimals = 2,      #2
    do_a_projection = False
    )